### Setting Up the Environment for OpenAI Integration

In this section, we are preparing our Python environment to integrate with OpenAI's services. This involves two key steps:

1. **Importing the OpenAI Library**: We begin by importing `OpenAI` from the `openai` package. This library is essential for interacting with OpenAI's API, which provides access to a range of AI-powered tools and models.

2. **Importing the OS Module**: Additionally, we import the `os` module, a standard Python library used for interacting with the operating system. This module will help us manage file paths, directories, and environment variables, which can be crucial for tasks like file handling and configuring API keys.

By setting up these imports, we lay the groundwork for utilizing OpenAI's capabilities in our Python scripts, ensuring that we can seamlessly integrate advanced AI features into our applications.

In [ ]:
from openai import OpenAI
import os

### Initializing the OpenAI Client

#### Important Note on API Key Configuration:

To successfully interact with OpenAI's API, an API key is required. This key acts as a secure method of authentication and allows your script to access OpenAI's services. There are two ways to provide this API key:

1. **Environment Variable (Recommended)**: For security and convenience, it is recommended to set the OpenAI API key as an environment variable. This method keeps the key secure and separate from your script. If you choose this approach, the `OpenAI()` client will automatically use the API key from the environment variable.
2. **Directly in the Code**: Alternatively, you can explicitly specify the API key in your code when creating the OpenAI client. However, this approach is less secure, especially if your code is shared or made public. If you prefer this method, you can initialize the client as follows (replace `YOUR_API_KEY` with your actual key from https://platform.openai.com/api-keys):

   ```python
   client = OpenAI(api_key='YOUR_API_KEY')
   ```

#### Caution:
It is crucial to keep your API key confidential to prevent unauthorized usage and potential security risks. Therefore, using the environment variable method is highly recommended.

In [ ]:
client = OpenAI() # create OpenAI client

### Finding and Validating Audio and Video Files in a Specified Folder

The updated function `find_audio_files_in_folder` is designed to locate various types of audio and video files (specifically mp3, mp4, mpeg, mpga, m4a, wav, and webm formats) within a given folder. This function not only identifies these files but also ensures each file is within a 25MB size limit, a crucial check for maintaining manageable file sizes for processing or upload constraints.

In [ ]:
def find_audio_files_in_folder(folder, sort_ascending=True):
    """
    Find all audio and video files (mp3, mp4, mpeg, mpga, m4a, wav, webm) in the specified folder,
    without searching in subfolders, and ensure file size is less than 25MB.

    :param folder: The folder path to search in.
    :param sort_ascending: Whether to sort the filenames in ascending order.
    :return: A list of full paths to the files in the folder, sorted as specified.
    :raises ValueError: If any file is larger than 25MB.
    """
    if not os.path.exists(folder):
        print(f"The folder {folder} does not exist.")
        return []

    allowed_extensions = ('.mp3', '.mp4', '.mpeg', '.mpga', '.m4a', '.wav', '.webm')
    files = []
    
    for file in os.listdir(folder):
        if file.endswith(allowed_extensions):
            full_path = os.path.join(folder, file)
            file_size = os.path.getsize(full_path) / (1024 * 1024) # size in MB
            if file_size > 25:
                raise ValueError(f"File uploads are currently limited to 25 MB. The file '{file}' exceeds this limit.")
            files.append(full_path)

    # Sort the files
    return sorted(files, reverse=not sort_ascending)


### Implementing Audio Transcription with OpenAI

In this section, we introduce a function designed to leverage OpenAI's advanced transcription services. The function, transcribe_using_openai, is crafted to transcribe audio content into text form. This functionality is particularly useful for converting spoken word recordings, such as podcasts, interviews, or lectures, into a written format.

In [ ]:
def transcribe_using_openai(audio_path, model="whisper-1"):
    """
    Transcribe an audio file using OpenAI's transcription service.

    :param audio_path: The file path of the audio file to transcribe.
    :param model: The model to use for transcription. Default is "whisper-1".
    :return: The transcribed text of the audio file.
    """
    # Open the audio file in read-binary mode
    # 'rb' mode is used because audio files are binary files
    audio_file = open(audio_path, "rb")

    # Use OpenAI's client to create a transcription of the audio file
    # The 'model' parameter specifies the transcription model to use
    # 'file' parameter takes the audio file object for processing
    # 'response_format' is set to "text" to get the transcription in text format
    transcript = client.audio.transcriptions.create(model=model, file=audio_file, response_format="text")

    # Return the transcription text
    return transcript


### Batch Transcription of Multiple Audio Files

We introduce the `transcribe_audio_files` function, a tool designed to facilitate the transcription of multiple audio files using OpenAI's transcription service. This function is especially useful when dealing with a collection of audio recordings that need to be converted into text format.

In [ ]:
def transcribe_audio_files(audio_files, model="whisper-1"):
    """
    Transcribe a list of audio files using the OpenAI transcription function.

    :param audio_files: List of paths to audio files.
    :param model: The model to be used for transcription.
    :return: List of transcripts for each audio file.
    """
    transcripts = []
    for audio_file in audio_files:
        try:
            transcript = transcribe_using_openai(audio_file, model)
            transcripts.append(transcript)
        except Exception as e:
            print(f"Error transcribing file {audio_file}: {e}")
            transcripts.append(None)  # Append None for any failed transcription

    return transcripts


### Saving Transcripts as Markdown Files

The `save_transcripts_to_md` function is designed to save a list of transcripts into a specified folder as Markdown (`.md`) files. This functionality is particularly useful for organizing and storing transcribed text in a structured and readable format.

In [ ]:
def save_transcripts_to_md(transcripts, folder="transcripts"):
    """
    Save a list of transcripts to a specified folder as .md files.

    :param transcripts: List of transcript texts.
    :param folder: The folder where the .md files will be saved.
    """
    # Ensure the folder exists, create if not
    if not os.path.exists(folder):
        os.makedirs(folder)

    # Find the starting index for naming to avoid overwriting
    existing_files = [f for f in os.listdir(folder) if f.startswith('transcript_') and f.endswith('.md')]
    if existing_files:
        last_index = max(int(f.split('_')[1].split('.')[0]) for f in existing_files)
    else:
        last_index = 0

    # Save each transcript as a .md file
    for i, transcript in enumerate(transcripts, start=last_index + 1):
        file_path = os.path.join(folder, f"transcript_{i}.md")
        with open(file_path, 'w') as file:
            file.write(transcript)

---

### Retrieving MP4 Files from an Audio Directory

This code snippet demonstrates the use of the `find_mp4_files_in_folder` function to list `.mp4` files from a specified "audio" directory. The output is printed to showcase the retrieved file paths.

In [ ]:
folder_containing_audio_files = "audio"
files = find_audio_files_in_folder(folder_containing_audio_files)
print(files)

### Transcribing Audio Files and Displaying Transcripts

This section of code utilizes the `transcribe_audio_files` function to transcribe a list of audio files. The resulting transcripts are then printed, allowing us to review the textual representation of each audio file's content.

In [ ]:
transcripts = transcribe_audio_files(files)
print(transcripts)

### Storing Transcripts as Markdown Files

This code executes the `save_transcripts_to_md` function, which is responsible for saving the generated transcripts into Markdown (`.md`) files within a default or specified folder. This process is essential for organizing and archiving the transcribed text data.

In [ ]:
save_transcripts_to_md(transcripts)